# Librerie

In [1]:
import pandas as pd

# Data Ingestion & Data Preparation

In [2]:
nomi=pd.read_csv('Nomi.csv')
nomi.loc[3]=('31567','Giorgia Stazzi, Remo Stazzi','Giacinto Oscano')
nomi.loc[4]=('98989','Giorgia Stazzi/Remo Stazzi','Giacinto Oscano')
nomi.loc[5]=('98987','Giorgia Ramarindo Mazziconi','Giorgia Stazzi')
nomi.loc[6]=('54543','Giorgia R. Mazziconi','Giorgia Stazzi')
nomi.loc[7]=('56564','Giorgia Ramarindo Mazziconi srl','Amica Chic Srl')
nomi.loc[8]=('81324','Giorgia R. Mazziconi srl','Amica Chic S.r.l.')
nomi.loc[9]=('78789','Giorgia R. Mazziconi srl','Amica Chic S. r. l.')
nomi.loc[10]=('12895','Giorgia R.Mazziconi','Giancarlo Mazziconi')

In [3]:
nomi

,COD_MOVIMENTO,ORDINANTE,BENEFICIARIO
0,12536,Aldo Vanni - Remo Stazzi,Stazzi Giorgia
1,65879,Aldo Vanni - Remo Stazzi,Stazzi Remo
2,14235,Aldo Vanni Remo Stazzi,Gloria Neppi
3,31567,"Giorgia Stazzi, Remo Stazzi",Giacinto Oscano
4,98989,Giorgia Stazzi/Remo Stazzi,Giacinto Oscano
5,98987,Giorgia Ramarindo Mazziconi,Giorgia Stazzi
6,54543,Giorgia R. Mazziconi,Giorgia Stazzi
7,56564,Giorgia Ramarindo Mazziconi srl,Amica Chic Srl
8,81324,Giorgia R. Mazziconi srl,Amica Chic S.r.l.
9,78789,Giorgia R. Mazziconi srl,Amica Chic S. r. l.


In [4]:
nomi.dtypes

COD_MOVIMENTO    object
ORDINANTE        object
BENEFICIARIO     object
dtype: object

In [5]:
#TEST
print 'Tette/unK'.lower().replace('/',' ').replace('t',' ')
print 'Tette/unK'.lower().replace('/',' ').replace('t',' ').split()
print list(set('Tette/unK'.lower().replace('/',' ').replace('t',' ').split()))

 e  e unk
['e', 'e', 'unk']
['unk', 'e']


In [6]:
nomi_femminili = pd.read_csv('nomi_femminili_fascia_eta_2013.csv',usecols=['nome','frequenza'])
nomi_maschili = pd.read_csv('nomi_maschili_fascia_eta_2013.csv',usecols=['nome','frequenza'])
print nomi_femminili.shape
print nomi_maschili.shape

(1302, 2)
(927, 2)


In [7]:
nomi_femminili.head()

,nome,frequenza
0,GIULIA,561
1,SOFIA,507
2,SARA,372
3,EMMA,276
4,MATILDE,262


In [8]:
#TEST
for i in nomi_maschili.index:
    if nomi_maschili['nome'][i]=='CIRO': print nomi_maschili[i:i+1]

     nome  frequenza
329  CIRO         26
     nome  frequenza
450  CIRO         29
     nome  frequenza
579  CIRO         24


In [9]:
nomi_femminili = pd.DataFrame(nomi_femminili.groupby(['nome'],as_index=False)['frequenza'].sum())
nomi_maschili = pd.DataFrame(nomi_maschili.groupby(['nome'],as_index=False)['frequenza'].sum())
print nomi_femminili.shape
print nomi_maschili.shape

(344, 2)
(217, 2)


In [10]:
#TEST
for i in nomi_maschili.index:
    if nomi_maschili['nome'][i]=='CIRO': print nomi_maschili[i:i+1]

    nome  frequenza
36  CIRO         79


In [11]:
nomi_italiani = nomi_femminili.merge(nomi_maschili, how='outer')
print nomi_italiani.shape
print nomi_femminili.shape[0]+nomi_maschili.shape[0]

(561, 2)
561


# Algoritmo innestato

## matchin_ordinante_beneficiario
### --- matchin_evaluation
#### ------ check_match
##### --------- name_cleaner

In [12]:
def name_cleaner (self, *common_names):
    self = self.lower().replace('/',' ').replace('-',' ').replace(',',' ').replace('.',' ')\
        .replace('srl',' ').replace('s r l',' ').replace('s  r  l',' ')\
        .replace('spa',' ').replace('s p a',' ').replace('s  p  a',' ')
    try:
        for each_common_names in common_names[0]:
            self = self.replace(each_common_names.lower(),' ')
    except: pass
    return self

In [13]:
#TEST
print name_cleaner('Tette/unK.spa s.r.L Vanessa')
print name_cleaner('Tette/unK.spa s.r.L Vanessa', nomi_italiani['nome'])

tette unk     vanessa
tette unk      


In [14]:
#TEST
nomi['ORDINANTE'][0]

'Aldo Vanni - Remo Stazzi'

In [15]:
def check_match (nameA, nameB, *common_names):
    ordinante = name_cleaner(nameA, *common_names)
    partial_ordinante = list(set(ordinante.split()))
    print partial_ordinante
    
    beneficiario = name_cleaner(nameB, *common_names)
    partial_beneficiario = list(set(beneficiario.split()))
    print partial_beneficiario
    
    names_union =partial_ordinante + partial_beneficiario
    wordcounts={}
    for single_word in names_union:
        wordcounts[single_word] = names_union.count(single_word)
    print wordcounts
    print wordcounts.values()
    print wordcounts.values().count(2)
    match = wordcounts.values().count(2)
    return match
    # Se match = 1 --> nome o cognome,
    # se match = 2 --> (nome e nome) o (nome e cognome) o (cognome e cognome)
    # se match = 3 --> forte possibilità se stesso

In [16]:
def matching_evaluation(data, output_column_name, nameA, nameB, *common_names):
    data[output_column_name] = 0
    for row in data.index:
        data[output_column_name][row] = check_match(data[nameA][row],data[nameB][row],*common_names)

In [ ]:
#TEST
matching_valuation(nomi, 'MATCH_NAMES_AND_SURNAMES','ORDINANTE', 'BENEFICIARIO')

In [ ]:
#TEST
nomi

In [ ]:
#TEST
matching_valuation(nomi, 'MATCH_SURNAMES','ORDINANTE', 'BENEFICIARIO', nomi_italiani['nome'])

In [ ]:
#TEST
nomi

In [ ]:
#TEST
def rinomina(nomissimi):
    nomissimi = nomissimi.rename(columns={'COD_MOVIMENTO':'COD'})
    return nomissimi
    
rinomina(nomi)
#se rinominago dentro funzione fuori dalla funzione non cambia nulla 

In [17]:
#TEST
nomi.columns[0]

'COD_MOVIMENTO'

In [18]:
#TEST
def funzio (a,b,*c,**d):
    print 'a -->',a
    print 'b -->',a
    print 'c -->',c
    print 'd -->',d
    print 'd(value) -->',d['nome_file']
    
funzio(1,'abv', nome_file = 'rabarbaro')

a --> 1
b --> 1
c --> ()
d --> {'nome_file': 'rabarbaro'}
d(value) --> rabarbaro


In [19]:
#TEST
nomi.to_csv('matching_ordinante_beneficiario.csv',columns=[nomi.columns[0],'ORDINANTE','BENEFICIARIO','MATCH'],index=False)

In [20]:
def matching_ordinante_beneficiario (data, output_column_name, nameA, nameB, *common_names, **output_file_csv):
    output_column_name = output_column_name.upper()
    matching_evaluation(data, output_column_name+'_NAMES_AND_SURNAMES', nameA, nameB)
    if common_names is not ():
        matching_evaluation(data, output_column_name+'_SURNAMES', nameA, nameB, *common_names)
        data[output_column_name] = data[output_column_name+'_NAMES_AND_SURNAMES']+data[output_column_name+'_SURNAMES']
    try:
        data.to_csv(output_file_csv['nome_file'], columns=[data.columns[0],nameA,nameB, data.columns[-1]],index=False)
    except: pass
    print 'COD -->',data.columns[0]    
    '''
    Se output_column_name >= 3 ->> probabile SE STESSO,
    se output_column_name = 2 ->> probabile PARENTE (stesso cognome),
    se output_column_name = 1 ->> li accomuna solo il nome di battesimo
    se output_column_name = 0 ->> no match
    '''

In [21]:
matching_ordinante_beneficiario(nomi, 'matcH','ORDINANTE', 'BENEFICIARIO', nomi_italiani['nome'],\
                                nome_file = 'matching_ordinante_beneficiario.csv')

['aldo', 'stazzi', 'vanni', 'remo']
['stazzi', 'giorgia']
{'aldo': 1, 'stazzi': 2, 'giorgia': 1, 'vanni': 1, 'remo': 1}
[1, 2, 1, 1, 1]
1
['aldo', 'stazzi', 'vanni', 'remo']
['stazzi', 'remo']
{'aldo': 1, 'stazzi': 2, 'vanni': 1, 'remo': 2}
[1, 2, 1, 2]
2
['aldo', 'stazzi', 'vanni', 'remo']
['neppi', 'gloria']
{'stazzi': 1, 'aldo': 1, 'remo': 1, 'neppi': 1, 'vanni': 1, 'gloria': 1}
[1, 1, 1, 1, 1, 1]
0
['stazzi', 'remo', 'giorgia']
['oscano', 'giacinto']
{'oscano': 1, 'stazzi': 1, 'giacinto': 1, 'giorgia': 1, 'remo': 1}
[1, 1, 1, 1, 1]
0
['stazzi', 'remo', 'giorgia']
['oscano', 'giacinto']
{'oscano': 1, 'stazzi': 1, 'giacinto': 1, 'giorgia': 1, 'remo': 1}
[1, 1, 1, 1, 1]
0
['ramarindo', 'mazziconi', 'giorgia']
['stazzi', 'giorgia']
{'stazzi': 1, 'ramarindo': 1, 'mazziconi': 1, 'giorgia': 2}
[1, 1, 1, 2]
1
['r', 'mazziconi', 'giorgia']
['stazzi', 'giorgia']
{'stazzi': 1, 'r': 1, 'mazziconi': 1, 'giorgia': 2}
[1, 1, 1, 2]
1
['ramarindo', 'mazziconi', 'giorgia']
['chic', 'amica']
{'chic':

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/kernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
#TEST
matching_ordinante_beneficiario(nomi, 'matcH','ORDINANTE', 'BENEFICIARIO', nomi_italiani['nome'])

In [ ]:
#TEST
matching_ordinante_beneficiario(nomi, 'matcH','ORDINANTE', 'BENEFICIARIO',\
                                nome_file = 'matching_ordinante_beneficiario.csv')

In [ ]:
#TEST
matching_ordinante_beneficiario(nomi, 'matcH','ORDINANTE', 'BENEFICIARIO')

In [22]:
nomi

,COD_MOVIMENTO,ORDINANTE,BENEFICIARIO,MATCH_NAMES_AND_SURNAMES,MATCH_SURNAMES,MATCH
0,12536,Aldo Vanni - Remo Stazzi,Stazzi Giorgia,1,1,2
1,65879,Aldo Vanni - Remo Stazzi,Stazzi Remo,2,1,3
2,14235,Aldo Vanni Remo Stazzi,Gloria Neppi,0,0,0
3,31567,"Giorgia Stazzi, Remo Stazzi",Giacinto Oscano,0,0,0
4,98989,Giorgia Stazzi/Remo Stazzi,Giacinto Oscano,0,0,0
5,98987,Giorgia Ramarindo Mazziconi,Giorgia Stazzi,1,0,1
6,54543,Giorgia R. Mazziconi,Giorgia Stazzi,1,0,1
7,56564,Giorgia Ramarindo Mazziconi srl,Amica Chic Srl,0,0,0
8,81324,Giorgia R. Mazziconi srl,Amica Chic S.r.l.,0,0,0
9,78789,Giorgia R. Mazziconi srl,Amica Chic S. r. l.,0,0,0


In [23]:
nomi.dtypes

COD_MOVIMENTO               object
ORDINANTE                   object
BENEFICIARIO                object
MATCH_NAMES_AND_SURNAMES     int64
MATCH_SURNAMES               int64
MATCH                        int64
dtype: object

In [24]:
pd.read_csv('matching_ordinante_beneficiario.csv')

,COD_MOVIMENTO,ORDINANTE,BENEFICIARIO,MATCH
0,12536,Aldo Vanni - Remo Stazzi,Stazzi Giorgia,2
1,65879,Aldo Vanni - Remo Stazzi,Stazzi Remo,3
2,14235,Aldo Vanni Remo Stazzi,Gloria Neppi,0
3,31567,"Giorgia Stazzi, Remo Stazzi",Giacinto Oscano,0
4,98989,Giorgia Stazzi/Remo Stazzi,Giacinto Oscano,0
5,98987,Giorgia Ramarindo Mazziconi,Giorgia Stazzi,1
6,54543,Giorgia R. Mazziconi,Giorgia Stazzi,1
7,56564,Giorgia Ramarindo Mazziconi srl,Amica Chic Srl,0
8,81324,Giorgia R. Mazziconi srl,Amica Chic S.r.l.,0
9,78789,Giorgia R. Mazziconi srl,Amica Chic S. r. l.,0


In [25]:
pd.read_csv('matching_ordinante_beneficiario.csv').dtypes

COD_MOVIMENTO     int64
ORDINANTE        object
BENEFICIARIO     object
MATCH             int64
dtype: object